<a href="https://colab.research.google.com/github/kaede-san0910/workshop-2022/blob/main/internship_2022_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# インターンシップ2022夏秋 ワークショップ
アプリケーションの開発業務を体感する、ソフトウェア開発コース

## Pythonの超基本

### 0-1.まず、雑に動かします。

In [ ]:
a = 1
b = 2

print(a+b)

### 0-2.標準入力を使う

In [ ]:
a = int(input("a = "))
b = int(input("b = "))

print("{} + {} = {}".format(a, b, a+b))

## Google Colabでカメラを使ってみる

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

import PIL
from io import BytesIO


def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

下記のコードを実行してみてください。  
"capture"でカメラから静止画が取得できます。

In [ ]:
from IPython.display import Image

filename = take_photo(filename='photo.jpg')
print('Saved to {}'.format(filename))
  
# Show the image which was just taken.
display(Image(filename))

## 今回のお題、MediaPipeを使ってみる

GoogleのオープンソースソフトウェアであるMediaPipeを使ってみます。  
プロジェクトページ  
https://google.github.io/mediapipe/  
Github  
https://github.com/google/mediapipe  

### 1-1. まず、ライセンスを確認する
オープンソースソフトウェアを使用する上で、初めに、ライセンスは確認するようにしてください。  
オープンソースソフトウェアは、ライセンス条項に従うことを条件に、自由に使って良いこととなっております。  
研究利用や商用利用が可能なものも多くあります。  
  
Google社は、下記のライセンスをもとに、MediaPipeの利用を認めております。  

### Apache-2.0 License
比較的利用条件の緩いライセンスで、商用利用可、改変・複製可、公開・再配布も可などが認められます。  
ただし、再配布する場合は、同じApache-2.0 Licenseで配布することが求められます。  
ライセンス条件は、下記を確認ください。  
https://www.apache.org/licenses/LICENSE-2.0

### 1-2. mediapipeをインストールをする。
python3がインストールされている状態であれば、次のコマンドでインストールは完了です。

In [ ]:
!pip install mediapipe

参考：https://google.github.io/mediapipe/getting_started/python.html

### 1-3. サンプルコードを動かしてみる
オープンソースソフトウェアは、基本的に、サンプルのソースコードも公開されております。  
ハンドトラッキングの機能を動かしてみます。  
https://google.github.io/mediapipe/solutions/hands

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# 画像を撮影する ←ココ追加！
filename = take_photo(filename='photo.jpg')

# For static images:
IMAGE_FILES = [filename]    #  ←ココ追加！撮影した画像を解析対象とする。
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print handedness and draw hand landmarks on the image.
    print('Handedness:', results.multi_handedness)
    if not results.multi_hand_landmarks:
      continue
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()
    for hand_landmarks in results.multi_hand_landmarks:
      print('hand_landmarks:', hand_landmarks)
      print(
          f'Index finger tip coordinates: (',
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
      )
      mp_drawing.draw_landmarks(
          annotated_image,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          mp_drawing_styles.get_default_hand_landmarks_style(),
          mp_drawing_styles.get_default_hand_connections_style())
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
    # Draw hand world landmarks.
    if not results.multi_hand_world_landmarks:
      continue
    for hand_world_landmarks in results.multi_hand_world_landmarks:
      mp_drawing.plot_landmarks(
        hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

In [ ]:
display(Image('/tmp/annotated_image' + str(idx) + '.png'))

### 1-4. 任意の手の座標情報を出力してみる。
mediapipeは、２つの座標系での座標情報を出力することが可能です。  

#### MULTI_HAND_LANDMARKS  
Collection of detected/tracked hands, where each hand is represented as a list of 21 hand landmarks and each landmark is composed of x, y and z. x and y are normalized to [0.0, 1.0] by the image width and height respectively. z represents the landmark depth with the depth at the wrist being the origin, and the smaller the value the closer the landmark is to the camera. The magnitude of z uses roughly the same scale as x.
  
#### MULTI_HAND_WORLD_LANDMARKS  
Collection of detected/tracked hands, where each hand is represented as a list of 21 hand landmarks in world coordinates. Each landmark is composed of x, y and z: real-world 3D coordinates in meters with the origin at the hand’s approximate geometric center.
  
それぞれの座標系における人差し指の座標情報を出力してみましょう。  



#### MULTI_HAND_LANDMARKS

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# 画像を撮影する ←ココ追加！
filename = take_photo(filename='photo.jpg')

# For static images:
IMAGE_FILES = [filename]    #  ←ココ追加！撮影した画像を解析対象とする。
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    annotated_image = image.copy()
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            annotated_image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

    # Print hand world landmarks.
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        print(
            f'x={hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x}, '
            f'y={hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y}, '
            f'z={hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z}'
        )

    # Flip the image horizontally for a selfie-view display.
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))

    # 画像を表示
    display(Image('/tmp/annotated_image' + str(idx) + '.png'))

#### MULTI_HAND_WORLD_LANDMARKS

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# 画像を撮影する ←ココ追加！
filename = take_photo(filename='photo.jpg')

# For static images:
IMAGE_FILES = [filename]    #  ←ココ追加！撮影した画像を解析対象とする。
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    annotated_image = image.copy()
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            annotated_image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

    # Print hand world landmarks.
    if results.multi_hand_world_landmarks:
      for hand_world_landmarks in results.multi_hand_world_landmarks:
        print(
            f'x={hand_world_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x}, '
            f'y={hand_world_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y}, '
            f'z={hand_world_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z}'
        )

    # Flip the image horizontally for a selfie-view display.
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))

    # 画像を表示
    display(Image('/tmp/annotated_image' + str(idx) + '.png'))

２つの座標系の使い分けはイメージできますか？  
例えば、ジェスチャー認識をさせる場合は、ワールド座標系を用いた方が良さそうです。

### 1-5. ジェスチャー認識をさせてみましょう。
先ほど取得した人差し指座標を用いて、０と１を認識させてみましょう。  
0と１の違いは、どんなパラメータを用いて区別することができるでしょうか？

#### 0 で撮影してみる

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# 画像を撮影する ←ココ追加！
filename = take_photo(filename='photo.jpg')

# For static images:
IMAGE_FILES = [filename]    #  ←ココ追加！撮影した画像を解析対象とする。
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    annotated_image = image.copy()
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            annotated_image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

    # Print hand world landmarks.
    if results.multi_hand_world_landmarks:
      for hand_world_landmarks in results.multi_hand_world_landmarks:

        x = hand_world_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
        y = hand_world_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
        z = hand_world_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z
        dist = (x**2 + y**2 + z**2)**(0.5)

        print(
            f'x={x}, '
            f'y={y}, '
            f'z={z}, '
            f'd={dist}'
        )

    # Flip the image horizontally for a selfie-view display.
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))

    # 画像を表示
    display(Image('/tmp/annotated_image' + str(idx) + '.png'))

#### 1で撮影してみる

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# 画像を撮影する ←ココ追加！
filename = take_photo(filename='photo.jpg')

# For static images:
IMAGE_FILES = [filename]    #  ←ココ追加！撮影した画像を解析対象とする。
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    annotated_image = image.copy()
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            annotated_image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

    # Print hand world landmarks.
    if results.multi_hand_world_landmarks:
      for hand_world_landmarks in results.multi_hand_world_landmarks:

        x = hand_world_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
        y = hand_world_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
        z = hand_world_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z
        dist = (x**2 + y**2 + z**2)**(0.5)

        print(
            f'x={x}, '
            f'y={y}, '
            f'z={z}, '
            f'd={dist}'
        )

    # Flip the image horizontally for a selfie-view display.
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))

    # 画像を表示
    display(Image('/tmp/annotated_image' + str(idx) + '.png'))

人差し指とワールド座標系の中心(0, 0, 0)の距離に着目すれば、  
１と０の区別はできそうだとわかりました。  
閾値を決めてジェスチャーを出力してみましょう！

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# 画像を撮影する ←ココ追加！
filename = take_photo(filename='photo.jpg')

# For static images:
IMAGE_FILES = [filename]    #  ←ココ追加！撮影した画像を解析対象とする。
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    annotated_image = image.copy()
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            annotated_image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

    # Print hand world landmarks.
    if results.multi_hand_world_landmarks:
      for hand_world_landmarks in results.multi_hand_world_landmarks:

        x = hand_world_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
        y = hand_world_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
        z = hand_world_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z
        dist = (x**2 + y**2 + z**2)**(0.5)

        gesture = "1" if dist > 0.06 else 0
         
        print(
            f'gesuture={gesture}, '
            f'dist={dist}'
        )

    # Flip the image horizontally for a selfie-view display.
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))

    # 画像を表示
    display(Image('/tmp/annotated_image' + str(idx) + '.png'))

### 1-6. ハンドトラッキングの理論を確認してみましょう。
下記に本件に関する文献のURLを貼っておきます。  
興味があれば、確認してみてください！  
https://arxiv.org/abs/2006.10214  
https://www.youtube.com/watch?v=I-UOrvxxXEk